### Modelling Notebook

In [40]:
# importing external libraries
from pathlib import Path
import os
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
set_config(transform_output = "pandas")

# Importing function to load data

# Making sure any changes are instantly added
%load_ext autoreload
%autoreload 2

from Modules.load_data import load_data
from Modules.preprocessing import missing_summary, merge_dfs, dollar_to_int, find_unique_values
from Modules.plotting import Plotter
from Modules.transforming import *

# Importing Pipelines
from Pipelines.Pipeline1 import Pipeline_for_exploration, Pipeline1, general_transformation_pipeline
from Pipelines.Pipeline2 import Pipeline_for_exploration2, Pipeline2, general_transformation_pipeline2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Obtaining absolute path to data folder

data_folder = str(Path(os.getcwd()) / "data")

In [3]:
# Loading the data from pickle

merged_df = pd.read_pickle(data_folder + "/merged_data.pkl")

In [16]:
# Applying the general transformation pipelines

merged_df1 = general_transformation_pipeline.fit_transform(merged_df)

merged_df2 = general_transformation_pipeline2.fit_transform(merged_df)

/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each e

In [44]:
# Following the transformation from pipeline1

# Setting the target column

#y = merged_df["target"]

# Test Train Split

X_train1, X_test1 = train_test_split(merged_df, test_size=0.25, random_state=42)

# Applying Pipeline1

X_train1 = Pipeline1.fit_transform(X_train1)
X_test1 = Pipeline1.transform(X_test1)

# Obtaining target column

y_train1 = X_train1["is_fraud"]
y_test1 = X_test1["is_fraud"]

# Dropping is fraud column

X_train1 = X_train1.drop(columns=["is_fraud"])
X_test1 = X_test1.drop(columns=["is_fraud"])

# Checkiing the proportion of positive values

print(f"% of fraudulent transactions in y_train: {y_train1.mean()}")
print(f"% of fraudulent transactions in y_test: {y_test1.mean()}")

# Setting params to do grid search over
numerical_parameter_grid = np.linspace(0,1,10)

params = [{
    "penalty": [None, "elasticnet"],
    "l1_ratio" : np.linspace(0,1,10).tolist(),
    "C": np.linspace(0,1,10).tolist(),
    "class_weight": [None, "balanced"]
}]

gs_logit = GridSearchCV(
    LogisticRegression(),
    param_grid=params,
    scoring = "f1",
    cv=5)

gs_logit.fit(X_train1, y_train1)

print(f"score on training set: {gs_logit.score(X_train1, y_train1)}")
print(f"score on testing set: {gs_logit.score(X_test1, y_test1)}")


/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each e

% of fraudulent transactions in y_train: 0.1302501302761855
% of fraudulent transactions in y_test: 0.1302954509926528


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio 

score on training set: 0.9030107092341887
score on testing set: 0.8424346737599749


In [45]:
print(f"score on training set: {gs_logit.score(X_train1, y_train1)}")
print(f"score on testing set: {gs_logit.score(X_test1, y_test1)}")

score on training set: 0.9030107092341887
score on testing set: 0.8424346737599749


In [46]:
# Following the transformation from pipeline2

# Setting the target column

#y = merged_df["target"]

# Test Train Split

X_train2, X_test2 = train_test_split(merged_df, test_size=0.25, random_state=42)

# Applying Pipeline1

X_train2 = Pipeline2.fit_transform(X_train2)
X_test2 = Pipeline2.transform(X_test2)

# Obtaining target column

y_train2 = X_train2["is_fraud"]
y_test2 = X_test2["is_fraud"]

# Dropping is fraud column

X_train2 = X_train2.drop(columns=["is_fraud"])
X_test2 = X_test2.drop(columns=["is_fraud"])

# Checkiing the proportion of positive values

print(f"% of fraudulent transactions in y_train: {y_train2.mean()}")
print(f"% of fraudulent transactions in y_test: {y_test2.mean()}")

# Setting params to do grid search over
numerical_parameter_grid = np.linspace(0,1,10)

params = [{
    "penalty": [None, "elasticnet"],
    "l1_ratio" : np.linspace(0,1,10).tolist(),
    "C": np.linspace(0,1,10).tolist(),
    "class_weight": [None, "balanced"]
}]

gs_logit = GridSearchCV(
    LogisticRegression(),
    param_grid=params,
    scoring = "f1",
    cv=5)

gs_logit.fit(X_train2, y_train2)

print(f"score on training set: {gs_logit.score(X_train2, y_train2)}")
print(f"score on testing set: {gs_logit.score(X_test2, y_test2)}")

/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:53: UserWarning: Could not infer format, so each e

% of fraudulent transactions in y_train: 0.1302501302761855
% of fraudulent transactions in y_test: 0.1302954509926528


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio 

score on training set: 0.923634898247028
score on testing set: 0.875253075844884


/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
1100 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/as3620/miniconda3/envs/FODS_coursework/lib/python3.13/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  Fil

In [47]:
print(f"score on training set: {gs_logit.score(X_train2, y_train2)}")
print(f"score on testing set: {gs_logit.score(X_test2, y_test2)}")

score on training set: 0.923634898247028
score on testing set: 0.875253075844884


As we can see the date transformations improve performance. We will now see what an effect increasing the size of the dataset has.

In [ ]:
# Following the transformation from pipeline2 and exploring different dataset sizes

# Setting the target column

#y = merged_df["target"]

# Test Train Split

X_train3, X_test3 = train_test_split(merged_df, test_size=0.25, random_state=42)

enhanced_pipeline2 = Pipeline(steps=[
    ("Pipeline2", Pipeline2),
    ("logit", LogisticRegression())
])

params = [{
    "Pipeline2__general_transformation_pipeline__Return_reduced_df__percentage": [0.01, 0.05, 0.1, 0.25, 0.5, 1],
    "logit__penalty": [None, "elasticnet"],
    "logit__l1_ratio" : np.linspace(0,1,10).tolist(),
    "logit__C": np.linspace(0,1,10).tolist(),
    "logit__class_weight": [None, "balanced"]
}]






# Applying Pipeline1

X_train2 = Pipeline2.fit_transform(X_train2)
X_test2 = Pipeline2.transform(X_test2)

# Obtaining target column

y_train2 = X_train2["is_fraud"]
y_test2 = X_test2["is_fraud"]

# Dropping is fraud column

X_train2 = X_train2.drop(columns=["is_fraud"])
X_test2 = X_test2.drop(columns=["is_fraud"])

# Checkiing the proportion of positive values

print(f"% of fraudulent transactions in y_train: {y_train2.mean()}")
print(f"% of fraudulent transactions in y_test: {y_test2.mean()}")

# Setting params to do grid search over
numerical_parameter_grid = np.linspace(0,1,10)

params = [{
    "penalty": [None, "elasticnet"],
    "l1_ratio" : np.linspace(0,1,10).tolist(),
    "C": np.linspace(0,1,10).tolist(),
    "class_weight": [None, "balanced"]
}]

gs_logit = GridSearchCV(
    LogisticRegression(),
    param_grid=params,
    scoring = "f1",
    cv=5)

gs_logit.fit(X_train2, y_train2)

print(f"score on training set: {gs_logit.score(X_train2, y_train2)}")
print(f"score on testing set: {gs_logit.score(X_test2, y_test2)}")